# Serve model deploy to vertex AI

references:
- https://medium.com/google-cloud/serving-machine-learning-models-with-google-vertex-ai-5d9644ededa3

## Setup environment

**Create a new virtual environment**

In [ ]:
# Create a virtual environment
python -m venv .venv

# Activate the virtual environment
# Source .venv/bin/activate
.venv\Scripts\activate

# (Optional) Deactivate conda environment
conda deactivate

# Upgrade pip
python -m pip install --upgrade pip

**Create a `requirements.txt`**

In [18]:
%%writefile requirements.txt

uvicorn[standard]==0.20.0
gunicorn==23.0.0
fastapi[standard]==0.115.0
scikit-learn==1.5.2
pytest==8.3.3
starlette==0.38.6
requests==2.32.3

Overwriting requirements.txt


In [ ]:
# Install dependencies
pip install --no-cache-dir -r requirements.txt

## Develop model

**Model object**

In [1]:
%%writefile model.py
import random

from sklearn.base import BaseEstimator, TransformerMixin


class SimpleSentimentModel(BaseEstimator, TransformerMixin):
    negative_length_threshold = 10
    positive_length_threshold = 30
    negative_ls = ["tiêu cực", "xấu", "tệ", "negative"]
    positive_ls = ["tích cực", "thích", "positive"]

    def __init__(self):
        pass

    def predict(self, text):
        text_lower = text.lower()
        if any(word in text_lower for word in self.negative_ls):
            return "negative", random.randrange(90, 100, step=1) / 100
        elif any(word in text_lower for word in self.positive_ls):
            return "positive", random.randrange(90, 100, step=1) / 100
        elif len(text) <= self.negative_length_threshold:
            return "negative", random.randrange(70, 90, step=1) / 100
        elif len(text) >= self.positive_length_threshold:
            return "positive", random.randrange(70, 90, step=1) / 100
        else:
            return "neutral", random.randrange(70, 95, step=1) / 100

Overwriting model.py


**training script**

In [2]:
%%writefile train.py
import os

import joblib
from model import SimpleSentimentModel  # Ensure this import is correct

if __name__ == "__main__":
    # Create an instance of the model
    model = SimpleSentimentModel()

    # Create directory if it doesn't exist
    model_dir = "models"
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    # Save the model using joblib, ensuring correct context
    joblib.dump(model, os.path.join(model_dir, "model.pkl"))
    print("Model saved successfully!")

Overwriting train.py


**Main app API**

In [4]:
%%writefile main.py
import os
from typing import List, Optional

import joblib
import uvicorn
from fastapi import FastAPI, HTTPException, Request
from model import SimpleSentimentModel  # noqa: F401
from pydantic import BaseModel

# Initialize FastAPI app
app = FastAPI(title="Sentiment Analysis API")

# Load the model with a safe file path
model_path = os.path.join("models", "model.pkl")
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}")

# Load the model, making sure SimpleSentimentModel is already imported
model = joblib.load(model_path)


# Pydantic models for prediction results
class Prediction(BaseModel):
    sentiment: str
    confidence: Optional[float]


class Predictions(BaseModel):
    predictions: List[Prediction]


# Function to process batch predictions
def get_prediction(instances):
    res = []
    for text in instances:
        sentiment, confidence = model.predict(text)
        res.append(Prediction(sentiment=sentiment, confidence=confidence))
    return Predictions(predictions=res)


# Health check route
@app.get("/health", status_code=200)
async def health():
    return {"health": "ok"}


# Prediction route to handle batch requests
@app.post(
    "/predict",
    response_model=Predictions,
    response_model_exclude_unset=True,
)
async def predict(request: Request):
    # Extract the JSON body from the request
    body = await request.json()

    # Validate the request body
    if "instances" not in body or not isinstance(body["instances"], list):
        raise HTTPException(
            status_code=400,
            detail="Invalid input format. 'instances' should be a list of texts.",
        )

    # Extract the instances (texts) from the request
    instances = [x["text"] for x in body["instances"]]

    # Get predictions
    output = get_prediction(instances)

    # Return the predictions
    return output


# Main function to run the FastAPI app
if __name__ == "__main__":
    uvicorn.run("main:app", host="0.0.0.0", port=8080)


Overwriting main.py


**Train model**

In [ ]:
python train.py

**Test app**

In [1]:
%%writefile test.py
from fastapi.testclient import TestClient

from main import app

client = TestClient(app=app)
base_url = ""


def test_health():
    response = client.get(f"{base_url}/health")
    assert response.status_code == 200
    assert response.json() == {"health": "ok"}
    print("pass: test_health")


def test_predict_item():
    response = client.post(
        f"{base_url}/predict",
        json={
            "instances": [
                {"text": "Cong hoa xa hoi chu nghia"},
                {"text": "doc lap"},
                {"text": "doc lap tich cuc"},
                {"text": "te doc"},
                {"text": "positive doc lap"},
            ]
        },
    )
    assert response.status_code == 200
    result = response.json()
    sentiments = [i["sentiment"] for i in result["predictions"]]
    assert sentiments == [
        "neutral",
        "negative",
        "neutral",
        "negative",
        "positive",
    ]
    print("pass: test_predict_item")


def test_predict_item_non_instance():
    response = client.post(
        f"{base_url}/predict",
        json={
            "instan": [
                {"text": "Cong hoa xa hoi chu nghia"},
                {"text": "doc lap"},
                {"text": "doc lap tich cuc"},
                {"text": "te doc"},
                {"text": "positive doc lap"},
            ]
        },
    )
    assert response.status_code == 400
    response.json() == {
        "detail": "Invalid input format. 'instances' should be a list of texts."
    }
    print("pass: test_predict_item_non_instance")


def test_predict_item_not_list():
    response = client.post(
        f"{base_url}/predict",
        json={"instan": {"text": "Cong hoa xa hoi chu nghia"}},
    )
    assert response.status_code == 400
    response.json() == {
        "detail": "Invalid input format. 'instances' should be a list of texts."
    }
    print("pass: test_predict_item_not_list")


if __name__ == "__main__":
    # test for running container
    import requests

    client = requests
    base_url = "http://127.0.0.1:8080"
    test_health()
    test_predict_item()
    test_predict_item_non_instance()
    test_predict_item_not_list()


Overwriting test.py


Run pytest in `cmd`

In [ ]:
pytest test.py

## Upload model image to Artifact Registry (GCP)

**Write Dockerfile**

In [2]:
%%writefile Dockerfile
FROM tiangolo/uvicorn-gunicorn:python3.11-slim

WORKDIR /app

COPY *.py ./
COPY models ./models
COPY requirements.txt ./requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r ./requirements.txt

EXPOSE 8080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

Overwriting Dockerfile


### Build & Push image bằng docker-command

**Build docker image**

In [ ]:
docker build -t asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api .

**Test image container**

Run container

In [ ]:
docker run --rm -p 8080:8080 asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api

Test container

In [ ]:
python test.py

**Push image to Artifact Registry (GCP)**

Authen GCP

In [ ]:
# docker login
gcloud auth login

Push Image

In [ ]:
docker push asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api

### Build & Push image bằng cloud-build

**Config cloud build**

In [4]:
%%writefile cloudbuild.yaml
steps:
# If training model in cloud and save model in GCS
# Assume Storage location of model: `gs://dev-aiml-model/models/sentiment`
# Download the model file in GCS to embed it into the image
  - name: 'gcr.io/cloud-builders/gsutil'
    args: ['cp', '-r', '${_MODEL_GCS_PATH}', './models']
    id: 'download-model'
  
  # Build the container image
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', '${_IMAGE_NAME}', '.']
    waitFor: ['download-model']
  
  # Push the container image to Artifact Registry
  - name: 'gcr.io/cloud-builders/docker'
    args: ['push', '${_IMAGE_NAME}']

images:
  - '${_IMAGE_NAME}'

# Substitution variables for flexibility
substitutions:
  _MODEL_GCS_PATH: 'gs://dev-aiml-model/models/sentiment'
  _IMAGE_NAME: 'asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api'

Overwriting cloudbuild.yaml


**Run cloud build**

In [ ]:
gcloud builds submit --config cloudbuild.yaml

## Serving model container - Vertex AI

**Container Requirement**

The docker container needs to follow the [container requirements](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements) defined by Google. The most important requirement is: 

---
**1. HTTP server**

Provide an `HTTP server` that listens for requests on `0.0.0.0` (must) on port `8080` (can be choice).

**HTTP Server** can be using:
- **Flask** , **FastAPI**, ...
- **TensorFlow Serving**, **TorchServe**, or **KServe Python Server**
- ...

[**HTTP Server** can be run by](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#server):
- [ENTRYPOINT instruction](https://docs.docker.com/engine/reference/builder/#entrypoint), [CMD instruction](https://docs.docker.com/engine/reference/builder/#cmd) or both in ***Dockerfile***
- Specify the `containerSpec.command` and `containerSpec.args` fields when you create your `Model` resource (override your container image's `ENTRYPOINT` and `CMD`)

---
**2. Health checks**

***a. startup probe*** (optional)

Check whether the container application has started. Nếu không cung cấp thì sẽ ko chạy, và ngay lập tức chạy ***health probe***

**Usecase**: Cần sử dụng cho các application cần có thời gian khởi động trong lần đầu tiên. Ví dụ, Nếu App cần thời gian để copy file model mới từ source bên ngoài container mỗi lần khởi động. Chúng ta có thể config ***startup probe*** để chờ cho đến khi việc copy hoàn thành và trả ra success


***b. health probe***

Check whether the container application is ready to accept traffic or receive request. Nếu không cung cấp path cụ thể thì Vertex sẽ sử dụng default path `/health`. Lưu ý là ***health probe*** chỉ chạy khi ***startup probe*** hoàn thành hoặc không được khai báo

Provide an `HTTP path` for **health checks** (default path `/health` with `HTTP GET`, it can be change in config): 
- Return a `200` within **10 seconds** after call when you’re container is ready to handle requests. Nội dung của phần phản hồi không quan trọng, vì Vertex AI sẽ bỏ qua chúng. Phản hồi này cho thấy rằng server đang hoạt động tốt (healthy). For example, if you need to load the model, ensure you return the `200` status code after the model is loaded.
- **If the server isn't ready to handle prediction requests**, nó không nên phản hồi yêu cầu trong vòng **10 giây**, hoặc phản hồi với bất kỳ mã trạng thái nào khác ngoài `200 OK`, ví dụ như `503 Service Unavailable`. Điều này cho thấy server đang không hoạt động tốt (unhealthy).

Nếu health probe nhận được phản hồi không tốt từ server (bao gồm cả trường hợp không có phản hồi trong vòng 10 giây), nó sẽ gửi thêm **tối đa 3 lần Health Checks nữa**, mỗi lần cách nhau **10 giây**. Trong khoảng thời gian này, Vertex AI vẫn coi server là hoạt động tốt. Nếu probe nhận được phản hồi tốt từ bất kỳ lần kiểm tra nào, nó sẽ quay lại **Health checks Process**. Tuy nhiên, **nếu probe nhận được 4 phản hồi không tốt liên tiếp**, Vertex AI sẽ dừng việc chuyển tiếp các yêu cầu dự đoán tới container đó (nếu mô hình được triển khai trên nhiều node, các yêu cầu sẽ được chuyển tới các container khác đang hoạt động tốt).

Vertex AI không khởi động lại container; thay vào đó, health probe vẫn sẽ tiếp tục gửi các yêu cầu kiểm tra định kỳ tới server không tốt. Nếu nhận được phản hồi tốt, container đó sẽ được đánh dấu là hoạt động tốt và bắt đầu nhận lại yêu cầu dự đoán.

**Hướng dẫn thực tế:**
- Trong nhiều trường hợp, **server HTTP** trong container của bạn có thể luôn phản hồi với mã trạng thái `200 OK` cho các yêu cầu kiểm tra sức khỏe. Nếu container tải các tài nguyên trước khi khởi động server, container sẽ không hoạt động tốt trong thời gian khởi động và bất kỳ lúc nào server HTTP gặp lỗi. Trong tất cả các thời gian khác, nó sẽ phản hồi là tốt.

- Đối với cấu hình phức tạp hơn, bạn có thể thiết kế server HTTP để cố tình phản hồi yêu cầu kiểm tra sức khỏe với trạng thái không tốt vào những thời điểm nhất định. Ví dụ, bạn có thể chặn lưu lượng dự đoán tới node trong một khoảng thời gian để container thực hiện bảo trì.

---
**3. Prediction**

Provide an `HTTP path` for **prediction** (default path `/predict` with `HTTP POST`, it can be change in config)
- `Content-Type: application/json` HTTP header

---

**4. Request body**

The request body is `JSON` format and must be 1.5 MB or smaller, need contain an `instances` key and can be has `parameters` :
```JSON
{
   "instances":[
      {
         "text":"DoiT is a great company."
      },
      {
         "text":"The beach was nice but overall the hotel was very bad."
      }
   ],
   "parameters": {}
}
```
- `instances` take is an array of **one or more JSON values** of any type. Each values represents an instance that you are providing a prediction for.
- `parameters` (optional if application is designed to require it) take a JSON object containing any parameters that your container requires to help serve predictions on the instances

---

**5. Response body**

The response body is `JSON` format and must be 1.5 MB or smaller, need contain an `predictions` key :
```JSON
{
 "predictions": [
   {
     "confidence": 0.9409326314926147,
     "sentiment": "POSITIVE"
   }
 ],
  "deployedModelId": <string>, # id of the Endpoint's DeployedModel
  "model": <string>, # The resource name of the Model
  "modelVersionId": <string>, # The version id of the Model
  "modelDisplayName": <string>, # The display name of the Model 
  "metadata": <value> # Request-level metadata returned by the model
}
```
- `predictions` take is an array of **one or more JSON values** representing the predictions that your container has generated for each of the INSTANCES in the corresponding request.

**6. Publishing requirements**
- Location: `asia-southeast1`
- [Permissions](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#permissions)
- [Environment variable](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#variables)

**7. Access model artifacts**

[Doc](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements#artifacts)

- **Nếu sử dụng pre-build container làm môi trường**: Thì phải cung cấp địa chỉ tại GCS (folder) chứa các file model được training sẽ chạy trên environment build từ pre-build container đó

- **Nếu sử dụng custom container làm môi trường**: Việc cung cấp địa chỉ GCS (folder) chứa các file trained model là optional, nó cần thiết trong việc sử dụng custom container chỉ làm environment runtime và ko chứa sẵn model, khi đó cần phải copy model vào để run trong environment đó. Còn nếu trong container chứa sẵn file model thì việc cung cấp địa chỉ folder (GCS) chứa file model là ko cần thiết

### Import to Model Registry (VertexAI)

Ta cần import Model Image từ **Artifact Registry** sang **Vertex AI** để có thể tận dụng các tính năng quản lý model AI của Vertex và serve được model.

**Chi phí sử dụng Model Registry (VertexAI)**: No Cost

Chỉ phát sinh chi phí khi sử dụng prediction: `Online prediction via Endpoint` hoặc `Batch Prediction`


1. Import bằng giao diện UI: [Doc](https://cloud.google.com/vertex-ai/docs/model-registry/import-model#custom-container)

<img src = "_image/import_model_registry.png">

2. Import model command

In [ ]:
gcloud ai models upload \
  --container-ports=8080 \
  --container-predict-route="/predict" \
  --container-health-route="/health" \
  --region=asia-southeast1 \
  --display-name=sentiment-fast-api \
  --container-image-uri=asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api

3. Sử dụng python SDK

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project='ext-pinetree-dw', location='asia-southeast1')
aiplatform.Model.upload(
          display_name="sentiment-fast-api",
          serving_container_image_uri="asia-southeast1-docker.pkg.dev/ext-pinetree-dw/dev-aiml-model/sentiment-fast-api",
          serving_container_predict_route="/predict",
          serving_container_health_route="/health",
          serving_container_ports=[8080]
    )

In [ ]:
# select model after import
model = aiplatform.Model("projects/ext-pinetree-dw/locations/asia-southeast1/models/<model_id>")

### Serve Vertex model by batch prediction

**Batch Prediction** là gửi request trực tiếp tới Model đã được imported vào **Model Registry** mà **Model này không cần deploy thành endpoint**. Khi đó data gửi vào trong 1 single request (có thể large size) và không yêu cầu reponse trả ra real-time.

Vertex AI Batch Prediction is made for **large datasets** that would **take too much time with an online prediction approach**. It provides a **scalable**, **serverless**, and **efficient service** for cases where you **don’t need an immediate response** (asynchronous).

#### Cost

Chi phí được tính bằng thời gian sử dụng [***resource per node hour***](https://cloud.google.com/vertex-ai/pricing#pred_apac), tổng của:
- **vCPU cost**: measured in vCPU hours
- **RAM cost**: measured in GB hours
- **GPU cost**: if either built into the machine or optionally configured, measured in GPU hours

#### Config Input data

##### [Input data format](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#input_data_requirements)

Support các loại input data sau:
- **JSONL (JSON Line)**: Linh hoạt và support việc parse object dưới dạng key-value (hữu ích in case muốn xác định chính xác feature-name : feature value muốn truyền vào model)
- **CSV**: lấy cột và pass them mà không sử dụng header (không lấy dòng đầu tiên). Tất cả các string value phải được đặt trong double-quote.
- **File-list**: được encoded as a base64 string that contain content in file.
- **TFRecord** : `tf-record` or `tf-record-gzip` read as a binary and encoded as base64
- **Bigquery**: is the same CSV. Cần phải chú trọng thứ tự cột truyền vào để xác định feature chính xác do không nhận thông tin feature name/column name.

> To use a BigQuery table as input, you must set [`InstanceConfig.instanceType`](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.batchPredictionJobs#instanceconfig) to `object` using the Vertex AI API.

Với mỗi 1 format, VertexAI sẽ tự động **transforms them into a JSON** before sending it to the model (check chi tiết phía dưới để xem phần định dạng sau khi convert từ raw data sang JSON format)

**Chú ý: Hầu hết việc fail batch prediction job xảy ra khi xuất hiện lỗi trong việc transform data to JSON format. Sau khi transform thì model lại nhận được được differenct format**



###### JSON Lines

[JSON Lines](https://jsonlines.org/) file store in a Cloud Storage bucket.

- JSON Lines file where each line contains an **array**:

    ***data is:***
    ```text
    [1, 2, 3, 4]
    [5, 6, 7, 8]
    ```

    ***convert to request body JSON format is:***
    ```json
    {"instances": [ [1, 2, 3, 4], [5, 6, 7, 8] ]}
    ```

- JSON Lines file where each line contains an **object**:

    ***data is:***
    ```text
    { "values": [1, 2, 3, 4], "key": 1 }
    { "values": [5, 6, 7, 8], "key": 2 }
    ```

    ***convert to request body JSON format is:***
    ```json
    {"instances": [
      { "values": [1, 2, 3, 4], "key": 1 },
      { "values": [5, 6, 7, 8], "key": 2 }
    ]}
    ```

> Note: For **PyTorch prebuilt containers**, Vertex AI wraps each instance in a `data` field before sending it to the prediction container. This is because TorchServe's default handlers expect each instance to be wrapped in a `data` field.



###### Bigquery

Vertex AI transforms each row from the table to a JSON instance.

***data table is:***

| column 1 | column 2 | column 3|
|----------|----------|---------|
| 1.0 | 3.0 | "Cat1"|
| 2.0 | 4.0 | "Cat2"|

***convert to request body JSON format is:***
```json
{"instances": [ [1.0,3.0,"cat1"], [2.0,4.0,"cat2"] ]}
```

**Convert datatype from bigquery to body request**

| **BigQuery Type** | **JSON Type** | **Example value**                  |
| ----------------- | ------------- | ---------------------------------- |
| String            | String        | "abc"                              |
| Integer           | Integer       | 1                                  |
| Float             | Float         | 1.2                                |
| Numeric           | Float         | 4925.000000000                     |
| Boolean           | Boolean       | true                               |
| TimeStamp         | String        | "2019-01-01 23:59:59.999999+00:00" |
| Date              | String        | "2018-12-31"                       |
| Time              | String        | "23:59:59.999999"                  |
| DateTime          | String        | "2019-01-01T00:00:00"              |
| Record            | Object        | { "A": 1,"B": 2}                   |
| Repeated Type     | Array[Type]   | [1, 2]                             |
| Nested Record     | Object        | {"A": {"a": 0}, "B": 1}            |



###### CSV

- One input instance per row in a CSV file. 
- The first row must be a header row. 
- Enclose all strings in double quotation marks ("). 
- Vertex AI doesn't accept cell values that contain newlines. 
- Non-quoted values are read as floating point numbers.

***data in csv:***

```text
"input1","input2","input3"
0.1,1.2,"cat1"
4.0,5.0,"cat2"
```

***convert to request body JSON format is:***
```json
{"instances": [ [0.1,1.2,"cat1"], [4.0,5.0,"cat2"] ]}
```


##### Partition data

**MapReduce** được sử dụng để chia data thành các replica, với yêu cầu data có khả năng partition:
- Automatically partitions **BigQuery**, **file list**, and **JSON lines** input (sử dụng mỗi partition thành 1 replica)
- **CSV** not able to partition
- **TFRecord** partition by file (mỗi file là 1 replica)

##### Filter and transformation data

- **Filter**: cấu hình subset field would be selected or exclude to input data

- **Transform**: config to `instanceType` is `array` or `object` format

Ví dụ: cấu hình [`instanceConfig`](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.batchPredictionJobs#instanceconfig) trong [`BatchPredictionJob`](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.batchPredictionJobs)

```json
{
  "name": "batchJob1",
  ...
  "instanceConfig": {
    "excludedFields":["customerId"] # remove 'customerID' column
    "instanceType":"object"
  }
}
```

```json
{
  "name": "batchJob1",
  ...
  "instanceConfig": {
    "includedFields": ["col1","col2"] # include col1 + col2
    "instanceType":"object"
  }
}
```

#### Config request a batch prediction


##### Build pipeline for batch job


[**1. Use gcloud command**](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#google-cloud-console)

In [3]:
BATCH_JOB_NAME = "sentiment-fast-api-batch-test"
MODEL_URI = model.resource_name
INPUT_FORMAT = "bigquery"
INPUT_URI = f"bq://{TABLE_ID}"
OUTPUT_FORMAT = "bigquery"
OUTPUT_URI = f"bq://{PROJECT_ID}"
MACHINE_TYPE = "n1-standard-2"
EXCLUDED_FIELDS = [ID_COLUMN_NAME]

# Create a list of columns to be included
ALL_COLUMNS = list(df_x_with_id.columns)
INCLUDED_FIELDS = ALL_COLUMNS.copy()
INCLUDED_FIELDS.remove(ID_COLUMN_NAME)

For Exclude fields

In [ ]:
# Create JSON body requests - Exclude fields
import json

request_with_excluded_fields = {
    "displayName": f"{BATCH_JOB_NAME}-excluded_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"excludedFields": EXCLUDED_FIELDS},
}

with open("request_with_excluded_fields.json", "w") as outfile:
    json.dump(request_with_excluded_fields, outfile)

In [ ]:
# send request to run job
! curl \
  -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -d @request_with_excluded_fields.json \
  https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/batchPredictionJobs

For Include fields

In [ ]:
# Create JSON body requests - Include fields
request_with_included_fields = {
    "displayName": f"{BATCH_JOB_NAME}-included_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"includedFields": INCLUDED_FIELDS},
}

with open("request_with_included_fields.json", "w") as outfile:
    json.dump(request_with_included_fields, outfile)

In [ ]:
# send request to run job
! curl \
  -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -d @request_with_included_fields.json \
  https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/batchPredictionJobs

[**2. Request by Python SDK**](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions#aiplatform_batch_predict_custom_trained-python_vertex_ai_sdk)

- Input data source: **GCS JSON Lines**
- Output data source: **GCS JSON Lines**

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="ext-pinetree-dw", location="asia-southeast1")

model_resource_name = "projects/ext-pinetree-dw/locations/asia-southeast1/models/<model_id>"
model = aiplatform.Model(model_resource_name)

model.batch_predict(
    job_display_name="fast-api-batch-test",
    gcs_source = ['gs://ext-pinetree-dw/data/test.jsonl',] ,   # list of jsonl files or gcs path folder
    gcs_destination = 'gs://ext-pinetree-dw/data/predictions',
    instances_format = "jsonl",
    predictions_format="jsonl",
    machine_type="n1-standard-2",
    accelerator_count=1,
    accelerator_type="NVIDIA_TESLA_K80",
    starting_replica_count=4,  # use only `starting_replica_count`, not use `max_replica_count`
    sync = True, # Set to False if you don't want to wait for the job to finish
)

- Input data source: **Bigquery**
- Output data source: **Bigquery**

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="ext-pinetree-dw", location="asia-southeast1")

model_resource_name = "projects/ext-pinetree-dw/locations/asia-southeast1/models/<model_id>"
model = aiplatform.Model(model_resource_name)

model.batch_predict(
    job_display_name="fast-api-batch-test",
    bigquery_source= "bq://ext-pinetree-dw.dataset.test_table",   # list of jsonl files or gcs path folder
    bigquery_destination_prefix = "bq://your-project.your_dataset.your_output_table",
    instances_format = "bigquery",
    predictions_format="bigquery",
    machine_type="n1-standard-2",
    accelerator_count=1,
    accelerator_type="NVIDIA_TESLA_K80",
    starting_replica_count=4,  # use only `starting_replica_count`, not use `max_replica_count`
    sync = True, # Set to False if you don't want to wait for the job to finish
)

##### Workflow

- **Batch**: Tất cả dữ liệu sẽ được chia thành nhiều nhóm (batches), mỗi batch với kích thước batch_size và lần lượt sẽ được đưa vào model để dự đoán.
- **Replica**: Replica là một bản sao của machine được cấu hình để thực hiện prediction. Hệ thống sẽ tạo ra nhiều replica giống nhau, 1 replica sẽ xử lý 1 batch nhất định song song với replica khác. Hệ thống sẽ tự động phân chia dữ liệu và gán cho từng replica để xử lý, điều này giúp tăng tốc đáng kể cho các công việc lớn.

Trong batch prediction, toàn bộ dữ liệu đầu vào được chia thành các batch và mỗi replica sẽ xử lý một số lượng batch nhất định. Hệ thống sử dụng một cơ chế tương tự như MapReduce để phân chia dữ liệu cho các replicas.

Quá trình phân chia này hoạt động hiệu quả nếu dữ liệu có thể được partitioned, tức là có thể chia thành các phần độc lập để các replicas xử lý song song. 
- Dữ liệu đầu vào từ BigQuery, danh sách file, và file JSON lines là những loại có thể tự động được phân chia. 
- Tuy nhiên, dữ liệu CSV không phải là định dạng phù hợp cho việc này do nó không dễ chia nhỏ một cách an toàn và hiệu quả.

**Recommendation**: ***specify the smallest machine type possible for your job and increase the number of replicas.*** Với mỗi 1 batch sẽ được chạy bởi 1 replica

**Thời gian chạy mỗi replica**

Để tối ưu chi phí, nên lựa chọn số replica sao cho thời gian chạy mỗi replica tối thiểu là 10 phút. Do billed được tính dựa trên per replica node hour, mà trong đó đã mất khoảng 5 phút để khởi động cho mỗi 1 replica, nếu thời gian chạy predition mỗi replica quá thấp thì chi phí chủ yếu chỉ là thời gian khởi tạo replica.

Cụ thể:

- Mỗi replica được phân chia một tập hợp các batch dữ liệu khi công việc dự đoán bắt đầu. Hệ thống sẽ tự động gán các batch này cho từng replica.
- Quá trình hoạt động liên tục: Sau khi một replica xử lý xong một batch, nó sẽ chuyển sang batch tiếp theo trong danh sách được gán cho nó mà không cần khởi động lại.
- Một replica chỉ dừng lại khi đã hoàn thành tất cả các batch mà nó được phân công hoặc khi toàn bộ công việc dự đoán kết thúc.

**Số lượng replicas**

- Với batch size <= 1 triệu bản ghi: Nên set `starting_replica_count` khoảng vài chục
- Với batch size >= 1 triệu bản ghi: Nên set `starting_replica_count` khoảng vài trăm
- Hoặc theo công thức: `Số replica` = `Số batches` / (`Số phút kỳ vọng hoàn thành batch prediction job` * `60` / `Số giây cần thiết để 1 replica hoàn thành được 1 batch`)
    > `Số batches` = `Số bản ghi` / `batch_size`

Tăng chỉ số `starting_replica_count` sẽ làm tăng nhanh quá trình hoàn thành prediction job, tuy nhiên cũng làm lãng phí nhiều thời gian cho việc khởi động replica mà không có tác dụng trong việc predict. Vertex fix cố định `starting_replica_count` là số replica sử dụng mà không có cơ chế autoscale trong quá trình chạy job.

**Cấu hình resource**
- Khác với online prediction, Batch prediction không autoscale resource
- Sử dụng `starting_replica_count`, không sử dụng đến tham số `max_replica_count` (như online prediction)
- Nếu sử dụng GPU, GPU machine types take more time to startup (10 minutes), nên khuyến nghị là kéo dài thời gian batch prediction để tận dụng được thời gian 1 lần khởi tạo (ít nhất 20 phút replica runtime) replica không chiếm đa số thời gian của việc chạy job.
- Nên điều chỉnh batch_size phù hợp tuỳ thuộc vào machine, data input and model type để đảm bảo về performance, giá trị ban đầu được set `batch_size = 64`

**Cách optimize prediction process time**
1. Dữ liệu sẽ được chia thành các batch với `batch_size = 64 records` mặc định, mỗi batch sẽ được gửi đến 1 replica dể thực hiện quá trình predict.
   - Tuỳ thuộc vào config matchine thì với mỗi batch sẽ chạy qua replica bị too large (khiến tràn memory --> fail batch) hoặc quá nhỏ khiến không tận dụng được sức mạnh của replica (mỗi replica phải chạy nhiều batch hơn --> tốn thời gian chạy --> tăng chi phí). Do đó nên test và điều chỉnh batch_size phù hợp.
2. Sử dụng GPU sẽ làm giảm thời gian processing nếu model được design để sử dụng GPU, bằng việc config `accelerator_type` và `accelerator_count`

**Schedule a batch prediction job**

Combine: **Cloud Scheduler** + **Cloud Function**, **Cloud Run**, or **Vertex AI Pipelines**

**Retries if error**

If your model or prediction container runs into an error, Google retries the prediction three times. This is applied to each batch.

**Monitoring**
- Không có hệ thống monitoring sẵn có cho batch prediction, chỉ có cho online prediction (nhưng tốn chi phí resource cho phần quản lý monitoring)

**Cách thức triển khai**
- Hãy test kết quả trước bằng việc triển khai theo phương pháp online prediction (deploy model to VertexAI Endpoint) để có thể ensure được model run success và check được output của model đúng, quá trình debug cũng dễ dàng hơn.
    >If your model already fails with online predictions, it will also fail with batch predictions.


#### Batch prediction output


##### Body response format

1. Nếu request body nhận được là **list of array**

***request contains:***

```json
{
  "instances": [
    [1, 2, 3, 4],
    [5, 6, 7, 8]
]}
```

***The prediction container returns:***

```json
{
  "predictions": [
    [0.1,0.9],
    [0.7,0.3]
  ],
}
```

***Then the JSON Lines output file is:***

```json
{ "instance": [1, 2, 3, 4], "prediction": [0.1,0.9]}
{ "instance": [5, 6, 7, 8], "prediction": [0.7,0.3]}
```

---
2. Nếu request body nhận được là **list of object**

***request contains:***

```json
{
  "instances": [
    {"values": [1, 2, 3, 4], "key": 1},
    {"values": [5, 6, 7, 8], "key": 2}
]}
```

***The prediction container returns:***

```json
{
  "predictions": [
    {"result":1},
    {"result":0}
  ],
}
```

  ***Then the JSON Lines output file is:***

  ```json
  { "instance": {"values": [1, 2, 3, 4], "key": 1}, "prediction": {"result":1}}
  { "instance": {"values": [5, 6, 7, 8], "key": 2}, "prediction": {"result":0}}
  ```

##### Output Result

Output có thể lưu ở **GCS** hoặc **Bigquery**

**1. Output Cloud Storage**

- Output có thể chứa cả `prediction.errors` and `prediction.results` files
- Mỗi batch prediction job thì sẽ có nhiều file output được tách ra do cơ chế shared và chạy replica
- The `prediction.results` sẽ luôn bao gồm 2 thành phần theo format ở mục trên: `instance` (the data we sent to the batch prediction) + `prediction` itself

`Json file`:
```json
{"instance": {"text": "DoiT is a great company."}, "prediction": {"sentiment": "POSITIVE", "confidence": 0.9409326314926147}}
{"instance": {"text": "The beach was nice but overall the hotel was very bad."}, "prediction": {"sentiment": "NEGATIVE", "confidence": 0.9964427351951599}}
```

>Unfortunately, removing the instance from the batch prediction response is not possible. Consider this if you have a large dataset and want to calculate the storage costs for the batch prediction.

**2. Output Bigquery**
- Kết quả của batch prediction sẽ **luôn luôn trên 1 bảng mới** : job create the `table` and or the `dataset` automatically
- Sẽ bao gồm cột `instance` + `prediction`, mỗi dòng tương ứng với 1 record.

### Serve Vertex model by online prediction (endpoint - realtime)

Bằng cách deploy model vào 1 endpoint (1 endpoint có thể chứa nhiều model cùng deploy vào). Khi đó, cách serve model được thực hiện tương tự với action call API và trả ra output một cách real-time.

**Limitations**
- Request Body and Response Body bị giới hạn với kích thước 1.5MB, có thể phải dùng giải pháp khác là lưu output tại GCS là trả ra kết quả là path tại GCS
- Sẽ luôn phải trả phí cho ít nhất 1 instance buộc phải chạy liên tục (để hứng chờ request), nếu số request tăng lên thì tự động autoscale theo config. Sẽ không phù hợp với việc sử dụng nhiều loại model và mỗi model phải sử dụng 1 endpoint. Thay vào đó, chuyển hướng sang sử dụng **Cloud Run**

**Cost**

Chi phí được tính theo used [***resource per node hour***](https://cloud.google.com/vertex-ai/pricing#pred_apac). 

#### Create model enpoint

Trước khi deploy model, ta cần tạo **endpoint** để từ đó sẽ đẩy online request và nhận lại real time prediction

**Endpoint** là nơi mà model có thể deploy vào đó. 1 **Endpoint** có thể chứa nhiều model version hoặc model type khác nhau hoặc có thể tái sử dụng cho model version mới hơn, tuỳ thuộc vào cách chia `%traffic` cho từng deployed model trong endpoint thì sẽ có tỷ lệ 1 request sẽ run bởi model nào --> Phù hợp cho việc testing và chuyển đổi giữa nhiều model

In [ ]:
gcloud ai endpoints create \
  --project=ext-pinetree-dw \
  --region=asia-southeast1 \
  --display-name=sentiment-fast-api-test

#### Deploy model to an enpoint

Deploy model to exited endpoint

In [ ]:
gcloud ai endpoints deploy-model <endpoint_id> \
  --project=ext-pinetree-dw \
  --region=asia-southeast1 \
  --model=<model_id> \
  --display-name=sentiment-fast-api-model-v1

#### Test prediction

In [ ]:
from google.cloud import aiplatform

project = "ext-pinetree-dw"
location = "asia-southeast1"
project_model_id = "234439745674"
endpoint_id = "7608484124768075776"

aiplatform.init(project=project, location=location)
endpoint = aiplatform.Endpoint(
    f"projects/{project_model_id}/locations/{location}/endpoints/{endpoint_id}"
)

instances = [
    {"text": "DoiT is a great company."},
    {"text": "The beach was nice but overall the hotel was very bad."},
]

prediction = endpoint.predict(instances=instances)
print(prediction)

## Serving model container - Cloud Run